In [79]:
# import libraries
import pandas as pd
import numpy as np
import os
import requests
# chemin du dataset
DATA_PATH = '../data/raw/'
DATA_PROCESSED = '../data/processed/'
    DATA_CSV = 'enedis_2022_13.csv'
# import du csv pour l'exploration
df = pd.read_csv(os.path.join(DATA_PROCESSED, DATA_CSV))

In [83]:
list_dpe=[]
def get_building_dpe(building_id):

    # Set the base URL for the ADEME DPE API
    base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines"

    # Set the query parameters for the API request
    params = {"q": building_id,
              "size": 10000}

    # Send the API request and get the response
    response = requests.get(base_url, params=params)

    # Check the status code of the response
    if response.status_code == 200:
        # If the request was successful, extract the DPE label from the response data
        data = response.json()
        if data['results']:
          dpe_label = data["results"][0]["Etiquette_DPE"]
          #print(data["total"])
          for i in range(0,len(data["results"])):
            if data["results"][i]["Etiquette_DPE"]:
              # todo fonction qui retourn la moyen du dpe
              #print(data["results"][i]["Etiquette_DPE"])
              list_dpe.append(data["results"][i]["Etiquette_DPE"])
            else:
              print(None)

          return list_dpe
    else:
        # If the request was not successful, raise an exception
        raise Exception(f"Error: {response.status_code}")

In [85]:
def convertir(list_antoine_dpe):
    # Dictionnaire pour convertir les lettres en chiffres
    dpe_to_num = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
    # Dictionnaire pour convertir les chiffres en lettres
    num_to_dpe = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G'}
    # Convertir chaque DPE en chiffre
    num_list = [dpe_to_num[dpe] for dpe in list_antoine_dpe]
    # Calculer la moyenne
    moyenne = sum(num_list) / len(num_list)
    # Arrondir la moyenne à l'entier le plus proche
    moyenne_arrondie = round(moyenne)
    # Convertir le score moyen en lettre
    score_dpe = num_to_dpe[moyenne_arrondie]
    return score_dpe

In [87]:
def get_average_dpe(iban):
    # Create an empty list to store the DPE labels
    dpe_labels = []
    # Iterate over the BAN IDs in the DataFrame
    for ban in iban:
        #print(ban)
        # Get the DPE label for the current BAN ID
        dpe = get_building_dpe(ban)
        # Add the DPE label to the list
        if dpe:
          dpe_labels.append(convertir(dpe))
        else:
          dpe_labels.append(None)
    # Add the DPE labels to the DataFrame
    df["Etique_DPE_API"] = dpe_labels
    return 

In [89]:
%%time
get_average_dpe(df['Identifiant__BAN'])

CPU times: total: 13.1 s
Wall time: 1min 20s


In [90]:
df

,annee,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,Identifiant__BAN,Etique_DPE_API
0,2022,14,112.413,8.029,6.720,13030_0290_00016,C
1,2022,12,44.089,3.674,6.720,13030_uc09nj_00097,B
2,2022,13,36.555,2.812,6.720,13030_uc09nj_00076,None
3,2022,11,72.700,6.609,6.076,13088_7920_00035,None
4,2022,25,45.478,1.819,9.568,13109_0029_00531,D
...,...,...,...,...,...,...,...
95,2022,16,31.235,1.952,4.234,13078_0652,C
96,2022,42,114.722,2.731,7.461,13091_6720_00034,C
97,2022,15,131.378,8.759,5.773,13092_2660,None
98,2022,49,154.557,3.154,4.740,13106_0033,C
